# For simple tests only

### Imports

In [1]:
import pandas as pd

### Bit of code

In [17]:
data = pd.read_csv("../data/raw/sentiment_data.csv", sep=";")


In [28]:
import pandas as pd

# Load the data from the text file
file_path = '../data/raw/Sentences_AllAgree.txt'  # Replace with your file path
with open(file_path, 'r', encoding='latin1') as file:
    lines = file.readlines()

# Split each line into text and sentiment
data = [line.rsplit('@', 1) for line in lines]

# Create a DataFrame
df = pd.DataFrame(data, columns=['Text', 'Sentiment'])

# Strip any extra whitespace
df['Text'] = df['Text'].str.strip()
df['Sentiment'] = df['Sentiment'].str.strip()

df.head()



,Text,Sentiment
0,"According to Gran , the company has no plans t...",neutral
1,"For the last quarter of 2010 , Componenta 's n...",positive
2,"In the third quarter of 2010 , net sales incre...",positive
3,Operating profit rose to EUR 13.1 mn from EUR ...,positive
4,"Operating profit totalled EUR 21.1 mn , up fro...",positive


In [32]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset
from transformers import BertModel, BertTokenizer


# Example dataset
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long),
        }

# Load data
df['Sentiment'] = LabelEncoder().fit_transform(df['Sentiment'])
texts = df['Text'].tolist()
labels = df['Sentiment'].tolist()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Prepare data loaders
X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.2, random_state=42)
train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_length=128)
val_dataset = SentimentDataset(X_val, y_val, tokenizer, max_length=128)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

# Define the model
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        output = self.drop(pooled_output)
        return self.out(output)

model = SentimentClassifier(n_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# Training loop
for epoch in range(3):  # Example epochs
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

# Save the model
torch.save(model.state_dict(), 'sentiment_model.pth')

# Predict for new text
def predict_sentiment(text, model, tokenizer):
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        max_length=128,
        truncation=True,
        padding='max_length',
        return_tensors='pt',
    )
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    with torch.no_grad():
        output = model(input_ids, attention_mask)
        predicted_class = torch.argmax(output, dim=1).item()
    return predicted_class

# Example usage
new_text = "This company is doing exceptionally well."
predicted_class = predict_sentiment(new_text, model, tokenizer)
print(f'Predicted Sentiment Class: {predicted_class}')


KeyboardInterrupt: 